In [1]:
# '''
# function ConnectButton(){
#     console.log("Connect pushed");
#     document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
# }

# setInterval(ConnectButton,60000);
# '''

# from google.colab import drive
# from os import chdir

# drive.mount('/content/drive')
# project_path = '/content/drive/MyDrive/Gproject/o-linked-site-prediction-feature-augment'
# chdir(project_path)

In [2]:
import os
import numpy as np
import pandas as pd

# set seed for the reproducible result
SEED = 42

In [3]:
data_dir = './data/integrated_features' # we will get names from the augmented proteins
protein_names = [x.split('.')[0] for x in os.listdir(data_dir) if x.split('.')[1] == 'csv'] # get protein name list to be processed for building machine learning models
print('the number of initial proteins:', len(protein_names))
print(protein_names[:10])

the number of initial proteins: 104
['A2ABU4', 'A2AHJ4', 'A2AKB9', 'A2AQ25', 'E9Q1P8', 'E9Q5G3', 'O08537', 'O09061', 'O35303', 'O70263']


## hyper parameter optimization by K-fold cross-validation

In [4]:
import tensorflow as tf
import keras
from keras.optimizers import Adam
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import KFold

import time
from IPython.display import clear_output

from ml_models import *

epochs = 1000
from keras.callbacks import EarlyStopping
patience = 30
callbacks = [EarlyStopping(patience=patience, restore_best_weights=True, monitor='val_loss')]

test_size = 0.2


### set initial parameters

In [5]:
from functions import *

In [6]:
initial_params = default_params.copy()
initial_params.update({
    'window_size'    : 10
    })

print('initial parameters')
for key, value in initial_params.items():
    print(f'{key:<14} : {value}')

search_space = {
    'rnn_layers'    : [2, 3, 4, 5],
    'dnn_layers'    : [2, 3, 4, 5],
}

initial parameters
rnn_layers     : 1
rnn_neurons    : 64
dnn_layers     : 3
dnn_neurons    : 64
activation     : softmax
loss           : categorical_crossentropy
metrics        : accuracy
optimizer_type : Adam
learning_rate  : 0.001
regularizer    : {'input': None, 'hidden': None, 'bias': None}
window_size    : 10


## set variables

In [7]:
model_type = 'LSTM_UP_BASIC_ONLY'

basic_columns = dict(pd.read_csv('./data/basic_columns.csv', header=0).values.squeeze())
print('# of augmented features:', len(basic_columns))

# set continuous input features
x_cts = [x for x in basic_columns.keys() if basic_columns.get(x) != 'object']
print('continuous features:')
display(dict(zip(range(len(x_cts)), x_cts)))

# set categorical input features
x_cat = [x for x in basic_columns.keys() if basic_columns.get(x) == 'object']
print('categorical features:')
print(dict(zip(range(len(x_cat)), x_cat)))

# input features
x_var = x_cts + x_cat

# set continuos output feature
y_cts = []

# set categorical output feature
y_cat = ['positivity']

# output features
y_var = y_cts + y_cat

# of augmented features: 13
continuous features:


{0: 'npa(-3,-1)', 1: 'ppo(-7,-5)', 2: 'n(S/T)', 3: 'flexibility', 4: 'p(1)'}

categorical features:
{0: 'side_-1', 1: 'side_1', 2: 'side_2', 3: 'side_3', 4: 'side_4', 5: 'side_5', 6: 'ss', 7: 'ss_angle'}


# build amino acid sequence dataset

### model training with K-fold

In [8]:
keras.utils.set_random_seed(SEED)
tf.config.experimental.enable_op_determinism()

model_update  = False

params = initial_params.copy()
MODELs = []
METRICs = []
METRIC_MEAN = []
model_id = 1
verbose = 0

data_x = []
data_y = []
for name in protein_names:
    data = pd.read_csv(f'./data/integrated_features/{name}.csv')
    ST_idx = np.where((data['residue'] == 'S') | (data['residue'] == 'T'))[0]
    
    # get X dataset
    x_onehot = get_onehots(data[x_var], columns = x_cat)
    x_features = list(x_onehot.columns)
    
    # get Y dataset
    y_onehot = get_onehots(data[y_var], columns = y_cat)
    y_labels = list(y_onehot.columns)
    
    for idx in ST_idx:
        window_x = np.array(get_window(x_onehot, idx, params['window_size']))
        label_y  = np.array(y_onehot.iloc[idx])
        
        data_x.append(window_x)
        data_y.append(label_y)
        
data_x = np.array(data_x)
data_y = np.array(data_y)

for param_name, space in search_space.items():
    for point in space:
        clear_output(wait=True)
        display(METRIC_MEAN)
        params[param_name] = point
        for key, value in search_space.items():
            print(f'{key:<14} : {value}')
        
        model = LSTM_CLS(data_x.shape[1], data_x.shape[-1], data_y.shape[-1], params) # I don't know why, but this row is helping producing the same training result of a neural network

        print('data x shape:', data_x.shape)
        print('data y shape:', data_y.shape)
        print('class y counts:', data_y.sum(0))
        print(f'class y ratio: {(data_y.sum(0)/len(data_y)).round(4)}')
        
        splitter = StratifiedShuffleSplit(n_splits = 1, test_size = test_size, random_state = SEED)
        train_idx, test_idx = list(splitter.split(data_x, data_y))[0]
        
        train_x = data_x[train_idx]
        train_y = data_y[train_idx]
        
        test_x = data_x[test_idx]
        test_y = data_y[test_idx]
        
        train_x, test_x = data_scaling(train_x, test_x)
        
        splitter_kf = KFold(n_splits = 5)
        for cv_idx, (train_idx_kf, test_idx_kf) in enumerate(splitter_kf.split(train_x, train_y)):
            train_x_kf, train_y_kf = train_x[train_idx_kf], train_y[train_idx_kf]
            test_x_kf, test_y_kf   = train_x[test_idx_kf],  train_y[test_idx_kf]
            
            # up-sample the training dataset
            train_x_kf, train_y_kf = upsample_data(train_x_kf, train_y_kf) 
            
            model = LSTM_CLS(data_x.shape[1], data_x.shape[-1], data_y.shape[-1], params) 
            model_name = name_model(f'{model_type}_KFOLD', params)
            
            model_folder  = f'./models/{model_name}_{data_x.shape}'
            if not os.path.exists(model_folder):
                os.makedirs(model_folder)
            model_path    = f'{model_folder}/{cv_idx}.h5'
            metric_path   = f'{model_folder}/{cv_idx}.csv'
            
            
            if not os.path.exists(model_path) or model_update:
                time_start = time.time()
                history = model.fit(train_x_kf, train_y_kf, verbose=verbose, 
                                    epochs = 10000, callbacks = callbacks,
                                    validation_data = (test_x_kf, test_y_kf))
                time_end = time.time()
                training_time = round((time_end - time_start)/60, 3)
                
                model.save_weights(model_path)
                
                test_loss, accuracy, precision, recall, f1 = metrics_classification(model, test_x, test_y)
                model_metrics = {
                    'model_id' : model_id,
                    'cv_idx'   : cv_idx,
                    **{f'train_{x}': train_x.shape[x] for x in range(len(train_x.shape))},
                    'train_y'     : train_y.shape[-1],
                    'test_size'   : test_x.shape[0],
                    **params,
                    'regularizer_input' : params['regularizer']['input'],
                    'regularizer_hidden' : params['regularizer']['hidden'],
                    'regularizer_bias' : params['regularizer']['bias'],
                    'training_time': training_time,
                    'test_loss': test_loss,
                    'accuracy': accuracy,
                    **{f'precision_{x}': precision[x] for x in range(len(precision))},
                    **{f'recall_{x}'   : recall[x] for x in range(len(recall))},
                    **{f'f1_{x}'       : f1[x] for x in range(len(f1))}}
                
                model_metrics = pd.DataFrame([model_metrics]).drop(['activation', 'loss', 'metrics', 'optimizer_type', 'regularizer'], axis=1)
                model_metrics.to_csv(metric_path, index=False)
                
            else:
                model.load_weights(model_path)
                model_metrics = pd.read_csv(metric_path, header=0)
                
            print(f'f1 score: {model_metrics.f1_1[0]}')
            
            model_metrics['model_id'] = model_id
            METRICs.append(model_metrics)
            MODELs.append(model)
        
        METRIC_MEAN = pd.concat(METRICs).groupby('model_id').mean()
        f1_best = METRIC_MEAN.sort_values('f1_1', ascending=False).iloc[0].f1_1
        print(f'best f1 score: {f1_best}')
        model_id += 1
    if param_name in ['learning_rate']: # for float-type parameters
        best_value = float(METRIC_MEAN.sort_values('f1_1', ascending=False).iloc[0][param_name])
        params[param_name] = best_value
        search_space[param_name][search_space[param_name].index(best_value)] = f"{best_value}"
    else: # for int-type parameters
        best_value = int(METRIC_MEAN.sort_values('f1_1', ascending=False).iloc[0][param_name])
        params[param_name] = best_value
        search_space[param_name][search_space[param_name].index(best_value)] = f"{best_value}"

clear_output(wait=True)
display(METRIC_MEAN)
for key, value in search_space.items():
    print(f'{key:<14} : {value}')


,cv_idx,train_0,train_1,train_2,train_y,test_size,rnn_layers,rnn_neurons,dnn_layers,dnn_neurons,...,regularizer_bias,training_time,test_loss,accuracy,precision_0,precision_1,recall_0,recall_1,f1_0,f1_1
model_id,,,,,,,,,,,,,,,,,,,,,
1,2.0,9222.0,21.0,53.0,2.0,2306.0,2.0,64.0,3.0,64.0,...,NaN,6.9628,0.349441,93.176,97.114,20.162,95.756,26.514,96.426,22.354
2,2.0,9222.0,21.0,53.0,2.0,2306.0,3.0,64.0,3.0,64.0,...,NaN,16.8202,0.294554,94.232,97.018,23.086,96.990,23.022,97.004,22.876
3,2.0,9222.0,21.0,53.0,2.0,2306.0,4.0,64.0,3.0,64.0,...,NaN,18.0114,0.242936,94.588,97.048,25.802,97.342,23.488,97.194,24.530
4,2.0,9222.0,21.0,53.0,2.0,2306.0,5.0,64.0,3.0,64.0,...,NaN,24.1002,0.263350,94.422,97.010,24.240,97.208,22.556,97.102,22.646
5,2.0,9222.0,21.0,53.0,2.0,2306.0,4.0,64.0,2.0,64.0,...,NaN,22.4548,0.260253,94.006,96.912,20.850,96.866,20.234,96.882,19.936
6,2.0,9222.0,21.0,53.0,2.0,2306.0,4.0,64.0,3.0,64.0,...,NaN,18.0114,0.242936,94.588,97.048,25.802,97.342,23.488,97.194,24.530
7,2.0,9222.0,21.0,53.0,2.0,2306.0,4.0,64.0,4.0,64.0,...,NaN,16.6764,0.241712,95.022,97.150,30.762,97.694,26.048,97.422,28.052
8,2.0,9222.0,21.0,53.0,2.0,2306.0,4.0,64.0,5.0,64.0,...,NaN,4.0052,0.238851,93.920,97.016,21.586,96.658,23.256,96.838,22.312


rnn_layers     : [2, 3, '4', 5]
dnn_layers     : [2, 3, '4', 5]


In [9]:
print('data x shape: ', data_x.shape)
print('data y shape: ', data_y.shape)
print('train x shape:', train_x.shape)
print('test  x shape:', test_y.shape)

data x shape:  (11528, 21, 53)
data y shape:  (11528, 2)
train x shape: (9222, 21, 53)
test  x shape: (2306, 2)


In [10]:
print('optimal parameters')
for key, value in params.items():
    print(f'{key:<14} : {value}')

optimal parameters
rnn_layers     : 4
rnn_neurons    : 64
dnn_layers     : 4
dnn_neurons    : 64
activation     : softmax
loss           : categorical_crossentropy
metrics        : accuracy
optimizer_type : Adam
learning_rate  : 0.001
regularizer    : {'input': None, 'hidden': None, 'bias': None}
window_size    : 10


## evaluate model's general performance through Monte-Carlo cross-validation

In [11]:
keras.utils.set_random_seed(SEED)
tf.config.experimental.enable_op_determinism()

model_update = False
            
MODELs = []
METRICs = []
METRIC_MEAN = []
verbose = 0
        
data_x = []
data_y = []

for name in protein_names:
    data = pd.read_csv(f'./data/integrated_features/{name}.csv')
    ST_idx = np.where((data['residue'] == 'S') | (data['residue'] == 'T'))[0]
    
    # get X dataset
    x_onehot = get_onehots(data[x_var], columns = x_cat)
    x_features = list(x_onehot.columns)
    
    # get Y dataset
    y_onehot = get_onehots(data[y_var], columns = y_cat)
    y_labels = list(y_onehot.columns)
    
    for idx in ST_idx:
        window_x = np.array(get_window(x_onehot, idx, params['window_size']))
        label_y  = np.array(y_onehot.iloc[idx])
        
        data_x.append(window_x)
        data_y.append(label_y)
        
data_x = np.array(data_x)
data_y = np.array(data_y)
model = LSTM_CLS(data_x.shape[1], data_x.shape[-1], data_y.shape[-1], params) # I don't know why, but this row is helping producing the same training result of a neural network

print('data x shape:', data_x.shape)
print('data y shape:', data_y.shape)
print('class y counts:', data_y.sum(0))
print(f'class y ratio: {(data_y.sum(0)/len(data_y)).round(4)}')

splitter = StratifiedShuffleSplit(n_splits = 5, test_size = test_size, random_state = SEED)
for cv_idx, (train_idx, test_idx) in enumerate(splitter.split(data_x, data_y)):
    train_x, train_y = data_x[train_idx], data_y[train_idx]
    train_x, train_y = upsample_data(train_x, train_y) # up-sample the training dataset
    test_x , test_y  = data_x[test_idx],  data_y[test_idx]
    
    train_x, test_x = data_scaling(train_x, test_x)
    
    model = LSTM_CLS(data_x.shape[1], data_x.shape[-1], data_y.shape[-1], params)
    model_name = name_model(f'{model_type}', params)
    
    
    model_folder  = f'./models/{model_name}_{data_x.shape}'
    if not os.path.exists(model_folder):
        os.makedirs(model_folder)
    model_path    = f'{model_folder}/{cv_idx}.h5'
    metric_path   = f'{model_folder}/{cv_idx}.csv'
    
    
    if not os.path.exists(model_path) or model_update:
        time_start = time.time()
        history = model.fit(train_x, train_y, verbose=verbose, 
                            epochs = 10000, callbacks = callbacks,
                            validation_split = test_size/(1-test_size))
        time_end = time.time()
        training_time = round((time_end - time_start)/60, 3)
        
        model.save_weights(model_path)
        
        test_loss, accuracy, precision, recall, f1 = metrics_classification(model, test_x, test_y)
        model_metrics = {
            'model_id' : model_id,
            'cv_idx'   : cv_idx,
            **{f'train_{x}': train_x.shape[x] for x in range(len(train_x.shape))},
            'train_y'     : train_y.shape[-1],
            'test_size'   : test_x.shape[0],
            **params,
            'regularizer_input' : params['regularizer']['input'],
            'regularizer_hidden' : params['regularizer']['hidden'],
            'regularizer_bias' : params['regularizer']['bias'],
            'training_time': training_time,
            'test_loss': test_loss,
            'accuracy': accuracy,
            **{f'precision_{x}': precision[x] for x in range(len(precision))},
            **{f'recall_{x}'   : recall[x] for x in range(len(recall))},
            **{f'f1_{x}'       : f1[x] for x in range(len(f1))}}
        model_metrics = pd.DataFrame([model_metrics]).drop(['activation', 'loss', 'metrics', 'optimizer_type', 'regularizer'], axis=1)
        model_metrics.to_csv(metric_path, index=False)
        
    else:
        model.load_weights(model_path)
        model_metrics = pd.read_csv(metric_path, header=0)
        
    print(f'f1 score: {model_metrics.f1_1[0]}')
    
    METRICs.append(model_metrics)
    MODELs.append(model)

METRICs = pd.concat(METRICs)
METRIC_MEAN = METRICs.groupby('model_id').mean()
METRIC_STD = METRICs.groupby('model_id').std()

data x shape: (11528, 21, 53)
data y shape: (11528, 2)
class y counts: [11100   428]
class y ratio: [0.9629 0.0371]
f1 score: 29.110000000000003
f1 score: 25.61
f1 score: 18.06
f1 score: 17.61
f1 score: 29.68


In [12]:
METRICs

,model_id,cv_idx,train_0,train_1,train_2,train_y,test_size,rnn_layers,rnn_neurons,dnn_layers,...,regularizer_bias,training_time,test_loss,accuracy,precision_0,precision_1,recall_0,recall_1,f1_0,f1_1
0,9,0,17760,21,53,2,2306,4,64,4,...,NaN,6.858,0.285783,95.14,97.18,31.94,97.79,26.74,97.49,29.11
0,9,1,17760,21,53,2,2306,4,64,4,...,NaN,4.204,0.292842,94.71,97.08,26.92,97.43,24.42,97.26,25.61
0,9,2,17760,21,53,2,2306,4,64,4,...,NaN,4.826,0.500383,94.49,96.78,20.29,97.52,16.28,97.15,18.06
0,9,3,17760,21,53,2,2306,4,64,4,...,NaN,29.634,0.408542,94.32,96.78,19.18,97.34,16.28,97.06,17.61
0,9,4,17760,21,53,2,2306,4,64,4,...,NaN,16.714,0.374377,95.27,97.18,33.33,97.93,26.74,97.55,29.68


In [13]:
METRIC_MEAN

,cv_idx,train_0,train_1,train_2,train_y,test_size,rnn_layers,rnn_neurons,dnn_layers,dnn_neurons,...,regularizer_bias,training_time,test_loss,accuracy,precision_0,precision_1,recall_0,recall_1,f1_0,f1_1
model_id,,,,,,,,,,,,,,,,,,,,,
9,2.0,17760.0,21.0,53.0,2.0,2306.0,4.0,64.0,4.0,64.0,...,NaN,12.4472,0.372385,94.786,97.0,26.332,97.602,22.092,97.302,24.014


In [14]:
METRIC_STD

,cv_idx,train_0,train_1,train_2,train_y,test_size,rnn_layers,rnn_neurons,dnn_layers,dnn_neurons,...,regularizer_bias,training_time,test_loss,accuracy,precision_0,precision_1,recall_0,recall_1,f1_0,f1_1
model_id,,,,,,,,,,,,,,,,,,,,,
9,1.581139,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,10.849699,0.088772,0.409304,0.204939,6.488919,0.248938,5.389482,0.212297,5.854087
